<a href="https://colab.research.google.com/github/longjae/raft_rag/blob/master/answer_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [24]:
import random
from datasets import load_dataset
from tqdm import tqdm
import openai
import re
import datasets
from huggingface_hub import login

In [3]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/77.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10434 [00:00<?, ? examples/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [4]:
# 임의 샘플링
df = df[000:500].reset_index(drop=True)

In [5]:
# 정답이 첫번째 문서에만 위치하지 않도록 문서를 섞는다.
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  documents = [context] + list(negative_samples)
  random.shuffle(documents)
  samples.append(documents)

In [6]:
# 프롬프트에 문서 넣는 연습
for idx, s in enumerate(samples[0]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. '장마전선'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 '장마'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 '창마전선'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. '창마' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 '창마' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 '창마' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기후 변화로 인해 '낙하 창마'라는 용어가 생겨났다. '낙하 창마'는 보통 8월 말에서 9월 초에 시작한다. 한반도에서 북태평양고기압이 완전히 끝난 뒤 '폭포창마'도 끝났다. 최근의 이 '폭포창마'는 보통의 '창마'보다 훨씬 더 큰 피해를 가져오는데, 왜냐하면 '폭포창마'는 단기간에 극도의 폭우가 집중적으로 쏟아지기 때문이다. 장마 순환 변화가 없을 경우 강수량이 증가할 것으로 예상되지만 비교적 완만한 이동이나 시기 변화는 동중국인, 한국, 일본 기후에 큰 영향을 미칠 수 있다.
---
문서2:부산시와 부산관광공사는 “2021 세계지구과학총회 유치위원회”(위원장 서울대 이상묵 교수)가 지난달 31일 “2021년 세계지구과학총회(MOCA 2021)”를 부산으로 유치했다고 밝혔다. 세계지구과학총회는 1969년부터 4년마다 60개국 1,500명이 참가하는 국제행사이다. 국제측지 및 지구물리연맹(IUGG) 산하 독립협회인 국제기상대기과학연합회, 국제해양물리과학연합회, 국제빙권과학협회의

In [7]:
# 빈 리스트 선언
user_prompts = []

# samples라는 5개의 문서가 섞여져있는 리스트들의 모음.
for sample in samples:
  user_prompt = ''
  # 각각의 sample(5개의 문서가 섞여있음)에 대해서 문서1, 문서2... 번호를 추가하여 하나의 문자열로 결합
  for idx, s in enumerate(sample):
    user_prompt = user_prompt + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [8]:
# 첫번째 데이터와 연관된 문서 5개 (1개엔 정답이 있을수도 있음.)
print(user_prompts[0])

문서1:열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. '장마전선'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 '장마'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 '창마전선'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. '창마' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 '창마' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 '창마' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기후 변화로 인해 '낙하 창마'라는 용어가 생겨났다. '낙하 창마'는 보통 8월 말에서 9월 초에 시작한다. 한반도에서 북태평양고기압이 완전히 끝난 뒤 '폭포창마'도 끝났다. 최근의 이 '폭포창마'는 보통의 '창마'보다 훨씬 더 큰 피해를 가져오는데, 왜냐하면 '폭포창마'는 단기간에 극도의 폭우가 집중적으로 쏟아지기 때문이다. 장마 순환 변화가 없을 경우 강수량이 증가할 것으로 예상되지만 비교적 완만한 이동이나 시기 변화는 동중국인, 한국, 일본 기후에 큰 영향을 미칠 수 있다.
---
문서2:부산시와 부산관광공사는 “2021 세계지구과학총회 유치위원회”(위원장 서울대 이상묵 교수)가 지난달 31일 “2021년 세계지구과학총회(MOCA 2021)”를 부산으로 유치했다고 밝혔다. 세계지구과학총회는 1969년부터 4년마다 60개국 1,500명이 참가하는 국제행사이다. 국제측지 및 지구물리연맹(IUGG) 산하 독립협회인 국제기상대기과학연합회, 국제해양물리과학연합회, 국제빙권과학협회의

In [9]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(df['question'].to_list(), user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [10]:
print(user_prompt_for_answer[0])

질문:북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
문서1:열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. '장마전선'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 '장마'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 '창마전선'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. '창마' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 '창마' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 '창마' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기후 변화로 인해 '낙하 창마'라는 용어가 생겨났다. '낙하 창마'는 보통 8월 말에서 9월 초에 시작한다. 한반도에서 북태평양고기압이 완전히 끝난 뒤 '폭포창마'도 끝났다. 최근의 이 '폭포창마'는 보통의 '창마'보다 훨씬 더 큰 피해를 가져오는데, 왜냐하면 '폭포창마'는 단기간에 극도의 폭우가 집중적으로 쏟아지기 때문이다. 장마 순환 변화가 없을 경우 강수량이 증가할 것으로 예상되지만 비교적 완만한 이동이나 시기 변화는 동중국인, 한국, 일본 기후에 큰 영향을 미칠 수 있다.
---
문서2:부산시와 부산관광공사는 “2021 세계지구과학총회 유치위원회”(위원장 서울대 이상묵 교수)가 지난달 31일 “2021년 세계지구과학총회(MOCA 2021)”를 부산으로 유치했다고 밝혔다. 세계지구과학총회는 1969년부터 4년마다 60개국 1,500명이 참가하는 국제행사이다. 국제측지 및 지구물리연맹(IUGG) 산하 독립

In [11]:
system_prompt = """당신은 검색된 문서로부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번, 5번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]], [[ref5]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 답변은 최대한 풍부하게 작성해주시기 바랍니다.
10. 검색 결과는 모두 독립적인 문서들이며 반드시 연관되어져 있거나 순서대로 이어지는 문서들이 아닙니다.
"""

In [13]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [17]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:14<00:00,  2.84s/it]


In [18]:
# 첫번째 샘플의 프롬프트
print(user_prompt_for_answer[0])

질문:북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
문서1:열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. '장마전선'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 '장마'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 '창마전선'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. '창마' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 '창마' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 '창마' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기후 변화로 인해 '낙하 창마'라는 용어가 생겨났다. '낙하 창마'는 보통 8월 말에서 9월 초에 시작한다. 한반도에서 북태평양고기압이 완전히 끝난 뒤 '폭포창마'도 끝났다. 최근의 이 '폭포창마'는 보통의 '창마'보다 훨씬 더 큰 피해를 가져오는데, 왜냐하면 '폭포창마'는 단기간에 극도의 폭우가 집중적으로 쏟아지기 때문이다. 장마 순환 변화가 없을 경우 강수량이 증가할 것으로 예상되지만 비교적 완만한 이동이나 시기 변화는 동중국인, 한국, 일본 기후에 큰 영향을 미칠 수 있다.
---
문서2:부산시와 부산관광공사는 “2021 세계지구과학총회 유치위원회”(위원장 서울대 이상묵 교수)가 지난달 31일 “2021년 세계지구과학총회(MOCA 2021)”를 부산으로 유치했다고 밝혔다. 세계지구과학총회는 1969년부터 4년마다 60개국 1,500명이 참가하는 국제행사이다. 국제측지 및 지구물리연맹(IUGG) 산하 독립

In [19]:
# 첫번째 샘플의 GPT-4o가 작성한 답변
result_lst[0]

'북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도에서 약 한 달가량 머무르는 것으로 알려져 있습니다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되며, 이 장마전선은 한반도 중남부를 오르내리며 비를 뿌립니다. 최근 30년간의 평균에 따르면 중부지방의 장마 기간은 약 32일로, 강수일수는 평균 17.2일에 달합니다[[ref4]].'

In [20]:
# 두번째 샘플의 프롬프트
print(user_prompt_for_answer[1])

질문:지능형 생산자동화 기반기술을 개발중인 스타트업은?
문서1:네스트(32억달러), 비츠(30억달러), 오큘러스VR(20억달러). 지난해 구글 애플 페이스북이 수십억달러를 들여 각각 인수한 실리콘밸리 대표 스타트업이다. 네스트는 가정용 온도조절기를 기반으로 가전기기를 묶어주는 스마트홈 플랫폼이다. 비츠와 오큘러스VR은 헤드셋 형태로 각각 음악과 가상현실 콘텐츠를 제공하는 웨어러블 기기다. 이들 기업의 공통점은 ‘하드웨어 스타트업’이라는 것이다.사물인터넷(IoT) 시대에 진입하면서 창의적인 아이디어를 바탕으로 IoT 제품을 만들어내는 하드웨어 스타트업이 늘고 있다. IoT는 스마트폰 애플리케이션(앱·응용프로그램)뿐만 아니라 인터넷에 연결해 유용한 서비스를 제공할 하드웨어가 필수이기 때문이다. 시장조사 업체 가트너는 “2017년까지 IoT 제품의 50%가 생긴 지 3년이 되지 않은 스타트업에 의해 만들어질 것”이라고 예측했다. 하지만 국내에는 하드웨어 스타트업을 위한 생태계가 갖춰지지 않아 관련 창업이 거의 일어나고 않고 있다.자본조달 장벽 높아스마트 팔찌 아키(Arki)를 개발 중인 스타트업 직토(Zikto)는 투자 유치에 어려움을 겪고 있다. 아키는 기존 웨어러블 밴드가 일반적으로 제공하던 운동량, 심박수 정보뿐만 아니라 사용자의 걸음걸이까지 분석·교정해주는 신개념 제품이다. 직토는 수만명의 일반인에게 제품을 소개하고 자금을 모으는 세계 최대 크라우드펀딩 사이트 킥스타터에서 16만5000달러(약 1억7800만원)를 끌어모으며 주목받았다. 하지만 한국에서 정식 투자를 받기는 녹록지 않다. 김경태 직토 대표는 “한국의 투자자들은 제품부터 가져오라고 하는데 하드웨어 스타트업은 시제품을 만드는 데까지만 해도 수억원의 선제 투자가 필요하다”며 “스마트폰 앱 업체와는 다른 하드웨어 스타트업의 생리를 이해해주는 투자자가 별로 없어 답답하다”고 말했다.하드웨어 스타트업은 대규모 초기 투자가 필요한 데 비해 자금 회수가 어렵다는 인식이 퍼졌다. 시제품을 만드는 데도 많은 돈이

In [21]:
# 두번째 샘플의 GPT-4o가 작성한 답변
result_lst[1]

'지능형 생산자동화 기반기술을 개발 중인 스타트업으로는 마키나락스가 있습니다. 이 회사는 딥러닝과 강화학습 기술을 활용하여 산업 공정을 효율화하는 AI 솔루션을 개발하고 있으며, 최근에는 다양한 제조업에서의 강화학습 적용 사례를 발표하고 있습니다. 마키나락스는 이러한 기술을 통해 제조 현장에서 발생하는 문제를 해결하기 위한 솔루션을 개발하고 있으며, 글로벌 성장 가능성을 인정받아 상당한 규모의 투자를 유치한 바 있습니다[[ref2]]. \n\n또한, 부산의 삼보테크놀로지도 지능형 생산자동화 기반기술을 개발하고 있습니다. 이들은 인공지능융합센서와 서보 이송 로봇을 이용한 자동용접 시스템을 개발하여 건설 분야의 생산성 및 안전성을 향상시키고자 하고 있습니다[[ref4]]. \n\n이 외에도 에어릭스는 스마트 팩토리 IoT 솔루션을 제공하여 산업 현장의 생산 관리를 개선하는 데 기여하고 있습니다[[ref3]]. 이러한 스타트업들은 지능형 생산자동화 기술을 통해 다양한 산업 분야에서 혁신을 이루고 있습니다.'

In [22]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 500/500 [17:53<00:00,  2.15s/it]


In [25]:
# 인용 문서 번호 추출
def extract_ref_numbers(text):
    # 정규표현식을 사용하여 [[ref숫자]] 패턴 찾기
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치 찾기
    matches = re.findall(pattern, text)

    # 문자열을 정수로 변환하고 중복 제거 후 정렬
    unique_numbers = sorted(set(map(int, matches)))

    return unique_numbers

In [26]:
result_lst[0]

'북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도에서 약 한 달가량 머무르는 것으로 알려져 있습니다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되며, 이 장마전선은 한반도 중남부를 오르내리며 비를 뿌립니다. 최근 30년간의 평균에 따르면 중부지방의 장마 기간은 약 32일로, 강수일수는 평균 17.2일에 달합니다[[ref4]].'

In [27]:
# 테스트
result = extract_ref_numbers(result_lst[0])
print(result)

[4]


In [28]:
# 모든 GPT-4o 답변으로부터 인용 문서 번호들을 추출한다.
extracted_ref_numbers = []
for sample in result_lst:
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [29]:
# 인용 번호 없는 샘플 찾아내기. 즉, GPT-4o 답변에서 인용 문서 번호가 없다는 것은 질문에 대한 답을 못 찾은 경우일 것이다.
for idx, extracted_ref_number in enumerate(extracted_ref_numbers):
  if len(extracted_ref_number) == 0:
    print(idx)
    print(result_lst[idx])
    print('--')

4
검색된 문서들에서는 정부가 환경과 관련하여 우선적으로 원조를 받고 있는 특정 지역이나 국가에 대한 정보는 제공되지 않았습니다. 따라서 이 질문에 대한 구체적인 답변을 찾을 수 없습니다.
--
6
검색된 문서들에서는 2015년에 열린 AAAI 학술대회에 논문을 개재한 사람의 소속 학교에 대한 정보가 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 찾을 수 없습니다.
--
23
검색된 문서들에서는 한국에 TAC 필름을 수출하는 국가에 대한 구체적인 정보가 포함되어 있지 않습니다. TAC 필름은 LCD TV와 스마트폰, 태블릿 PC 등에 사용되는 핵심 소재로, 한국은 이 필름을 일본에서 전량 수입하고 있다는 내용이 있습니다. 그러나 한국에 TAC 필름을 수출하는 다른 국가에 대한 정보는 확인할 수 없었습니다. 따라서 질문에 대한 답변을 찾을 수 없습니다.
--
61
주어진 문서들에서는 사장이 부하직원에게 행사할 수 있는 권리에 대한 구체적인 내용이 포함되어 있지 않습니다. 따라서 사장이 부하직원에게 행사할 수 있는 권리에 대한 답변을 제공할 수 없습니다.
--
64
검색된 문서들에서는 돌비 협력 작품 중 4K 화질을 지원하지 않는 특정 작품에 대한 정보가 포함되어 있지 않습니다. 따라서 이 질문에 대한 답변을 찾을 수 없습니다.
--
69
검색된 문서들에서는 미끄럼 방지용 논슬립패드가 없는 제품의 가격에 대한 정보가 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 찾을 수 없습니다.
--
70
검색된 문서들에서는 요자파트가 처음으로 근무했던 은행의 위치에 대한 정보가 포함되어 있지 않습니다. 따라서 해당 질문에 대한 답변을 찾을 수 없습니다.
--
129
콘스탄틴이 죽은 후 만들어진 조각의 수에 대한 정보는 문서에서 명확히 언급되지 않았습니다. 그러나 문서1에서는 콘스탄틴 브랑쿠시의 작품 중 일부가 사후에 제작되었다는 논란이 있음을 언급하고 있습니다. 이와 관련하여, 사후에 제작된 조각이 몇 점인지에 대한 구체적인 수치는 제공되지 않았습니다. 

In [30]:
print(df.loc[479])

title                                            서울대, 올해 등록금 0.3% 내린다
news_category                                                    교육산업
source                                                       hankyung
context             서울대가 2015학년도 학부 및 대학원 등록금을 전년보다 0.3% 내리기로 했다. ...
question                                           2년째 등록금이 동일한 지방대는?
question_type                                                       1
is_impossible                                                   False
answer_text                                                       경북대
answer_start                                                       46
negative_samples    [“4년째 재정적자가 누적되고 있습니다. 오죽하면 정부 재정 지원을 포기하면서까지 ...
Name: 479, dtype: object


In [31]:
print(user_prompt_for_answer[479])

질문:2년째 등록금이 동일한 지방대는?
문서1:“4년째 재정적자가 누적되고 있습니다. 오죽하면 정부 재정 지원을 포기하면서까지 등록금을 올리려고 했겠습니까.”(정진영 경희대 대외협력부총장) 등록금 동결 기조 속에서 ‘3.7% 인상안’을 내놓아 관심을 모았던 경희대가 2주 만에 백지화를 결정했다. 경희대는 최근 열린 등록금심의위원회에서 총학생회 측의 등록금 동결 요구를 수용했다고 13일 밝혔다. 정진영 부총장은 “정부의 반값 등록금 정책을 고려해 3년 연속 동결을 결정했다”며 “재정 문제는 운영 효율화 등으로 해결하겠다”고 말했다. 경희대는 지난달 28일 올해 등록금을 3.7% 인상하겠다고 발표했다. 등록금을 동결·인하한 대학이 자체적으로 마련한 장학금에 비례해 지원받는 국가장학금 2유형도 신청하지 않았다. 지난해 경희대는 국가장학금 2유형 18억원을 포함해 516억원의 정부 재정 지원을 확보했다. 연간 운영수익(기업의 매출에 해당) 4146억원의 12.4%에 해당한다. 반면 등록금 수입 2895억원에서 3.7%를 올리면 107억원가량을 더 받을 수 있다. 국가장학금 2유형을 유치하기 위해 지급했던 장학금 43억원을 더하면 150억원을 확보할 수 있다. 경희대 측은 “정부 지원 500억원을 포기하더라도 장기적인 재정 정상화를 위해 등록금을 제대로 받는 것이 낫다고 판단했다”고 설명했다. 이 대학의 평균 등록금은 763만원으로 연세대(856만원) 한양대(838만원) 성균관대(833만원) 등보다 낮은 편이다. 3~4년 연속 등록금을 동결한 다른 사립대들도 경희대를 유심히 지켜봤다. 그러나 학생들의 항의가 거세지자 인상 계획을 철회했다. 2011년 고등교육법 개정으로 등록금심의위는 정원의 10분의 3 이상을 학생 위원으로 구성해야 한다. 정부의 압박이 상당히 작용했다는 분석도 있다. 교육부 고위 관계자는 “올해가 박근혜 정부 출범 이후 처음으로 등록금을 결정하는 해인 데다 내년에 반값 등록금 정책이 완성될 예정이기 때문에 ‘올해 등록금을 올리는 대학이 있어선 안 된다

In [32]:
df['search_result'] = samples

In [33]:
df['answer'] = result_lst

In [34]:
df['extracted_ref_numbers'] = extracted_ref_numbers

In [35]:
# 인용 실패한 데이터 제외
cited_df = df[~(df['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

In [36]:
print('답변이 존재하는 데이터:', len(cited_df))

답변이 존재하는 데이터: 477


In [37]:
cited_df[['question', 'search_result', 'answer', 'extracted_ref_numbers']]

,question,search_result,answer,extracted_ref_numbers
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,[열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성...,북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도에서 약 한 달가량...,[4]
1,지능형 생산자동화 기반기술을 개발중인 스타트업은?,"[네스트(32억달러), 비츠(30억달러), 오큘러스VR(20억달러). 지난해 구글 ...",지능형 생산자동화 기반기술을 개발 중인 스타트업으로는 마키나락스가 있습니다. 이 회...,"[2, 3, 4]"
2,개막전에서 3안타 2실점을 기록해서 패한 선수는?,"[;한큐 - 사카모토 도시조\n* 타석에서의 상황 : 헛스윙, 볼, 헛스윙, 파울,...",개막전에서 3안타 2실점을 기록하여 패한 선수는 사이타마 세이부 라이온스의 와쿠이 ...,[2]
3,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[‘짜파구리’로 시작된 ‘나만의 레시피’ 열풍이 올해 식품시장을 주도한 것으로 나타...,컵라면 매출에서 불닭볶음면을 이긴 상품은 세븐일레븐의 '교동반점 짬뽕'입니다. 이 ...,[3]
4,에티하드 웰니스 프로그램의 일환으로 위생에 관한 정보를 제공하는 것은 누구인가?,[문종수 웨이웨어러블 대표(30)는 어릴 때부터 체질상 몸무게 변화가 심했다. 몸이...,에티하드 웰니스 프로그램의 일환으로 위생에 관한 정보를 제공하는 것은 에티하드항공의...,[3]
...,...,...,...,...
472,가끔씩 북동부 지역에 살고있는 오리너구리들을 사냥하는 포식자는 누구인가?,[오리너구리가 맨 처음 유럽 자연과학자들에게 알려졌을 때 본 동물이 난생인지 태생인...,"북동부 지역에 서식하는 오리너구리의 포식자는 여러 종류가 있으며, 주요 포식자로는 ...",[5]
473,윌리엄 헤이 칼드웰이 오리너구리에 대해 스코틀랜드에 보낸 메시지는?,[오리너구리가 맨 처음 유럽 자연과학자들에게 알려졌을 때 본 동물이 난생인지 태생인...,"윌리엄 헤이 칼드웰은 오리너구리에 대한 중요한 발견을 하였고, 그 결과를 스코틀랜드...",[1]
474,5종의 단공류 중 네가지의 종류를 차지하는 동물은?,[조르주 퀴비에와 에티엔 조프루아 생틸레르는 19세기 초 프랑스의 저명한 동물학자로...,5종의 단공류 중 네 가지의 종류를 차지하는 동물은 오리너구리와 가시두더지입니다. ...,[4]
475,오리너구리가 전기를 가장 잘 느끼는 부위는?,"[오리너구리는 반수생 동물로, 동부 호주 전체에 분포하며 강이나 연못, 소택지, 또...",오리너구리가 전기를 가장 잘 느끼는 부위는 부리입니다. 오리너구리의 전기수용기는 부...,[4]


In [40]:
from google.colab import userdata
HF_API_KEY = userdata.get('HF_WRITE_API_KEY')
login(HF_API_KEY)

In [41]:
dataset = datasets.Dataset.from_pandas(cited_df)

In [42]:
dataset.push_to_hub("klue-mrc-original-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/longjae/klue-mrc-original-question/commit/f3164f5bae082739bfa211f65a862886fc30602c', commit_message='Upload dataset', commit_description='', oid='f3164f5bae082739bfa211f65a862886fc30602c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/longjae/klue-mrc-original-question', endpoint='https://huggingface.co', repo_type='dataset', repo_id='longjae/klue-mrc-original-question'), pr_revision=None, pr_num=None)

In [43]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [44]:
df = df[1500:1800].reset_index(drop=True)

In [45]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples
0,어린이날 실수요자 끌기 … TV·車까지 경품 푸짐,부동산,hankyung,"‘자동차, 곡면 TV, 레고 세트….’ 가정의 달 5월을 맞아 아파트를 분양 중인 ...",유아용 전동자동차와 장난감 레고 세트를 경품 추첨으로 주는 아파트는?,2,False,‘김포한강신도시 반도유보라4차’,372,[유통업계가 황금 연휴(1~5일)로 시작하는 ‘가정의 달’ 5월을 맞아 대규모 세일...
1,이혼 건수 늘었는데...소송 줄어든 이유는,종합,hankyung,최근 들어 이혼 건수는 소폭 늘어나고 있지만 이혼 소송은 오히려 줄어들고 있는 것으...,2009년 이혼 소송 접수 건수는 몇 건인가?,1,False,4만7907건,318,[여성의 경제력과 사회적 지위가 높아지면서 이혼이나 사별(死別) 경험이 있는 여성의...
2,대기업 규제해도 동네 빵집 안늘어,생활경제,hankyung,"대기업 계열 빵집 확장을 제한한 결과, 중소 프랜차이즈는 약진하는 반면 정작 ‘동네...",규제로 인해 어떤 종류의 매장이 증가했는가?,1,False,중소 프랜차이즈,22,[주요 유통 및 필수 소비재 업체들이 여기저기서 허리띠를 졸라매고 있다. 무분별하게...
3,조텍컵 오버워치 아시아 커뮤니티 참가 모집,게임,acrofan,지포스 그래픽카드 전문 기업 조텍(ZOTAC Technology)이 남녀노소 누구나...,조텍컵에서 입상하면 어떤 플랫폼을 통해 상금을 받을 수 있는가?,1,False,페이팔,484,"[조텍코리아(ZOTAC Korea, 대표이사 김성표)가 엔비디아와 함께 차세대 그래..."
4,"(주)코멘토, 서울과기대에 직무부트캠프 제공",사회,acrofan,(주)코멘토가 유망 분야 직무에 대한 직무부트캠프를 서울과학기술대학(총장 이동훈)과...,직무부트캠프를 제공하는 회사를 운영하는 사람의 이름은?,2,False,이재성,595,[“구글이 꿈의 직장? 실력 없으면 못 버틴다”최근 구글 본사에서 일하는 한국인 개...
...,...,...,...,...,...,...,...,...,...,...
295,경단녀에게 이만한 직장 없죠”,종합,hankyung,“영어교육학과를 졸업했지만 기간제 교사로 일하는 것보다 하루 6시간 방과후학교 업무...,여선영씨와 매일 출퇴근을 함께 하는 사람은?,1,False,아들,232,[남덕우 전 국무총리 빈소가 차려진 서울 일원동 삼성서울병원 장례식장엔 19일 아침...
296,침팬지도 나폴레옹·마키아벨리 못지않게 전략 짠다,책마을,hankyung,현대 사회에서 전략은 일상 용어다. 국가나 대기업이 사활을 걸 정도로 중요한 결정일...,전략의 역사'의 저자는 본래 어떤 학문을 연구하는 전문가인가?,1,False,전쟁연구학,223,[≪전쟁론≫ 이전에 전쟁 이론은 대부분 전략 및 전술 중심의 테마로 구성되었으나 ≪...
297,대통령 주치의 이병석 교수,종합,hankyung,박근혜 대통령의 주치의로 이병석 연세대 의대 산부인과 교수(강남세브란스병원장·사진)...,대통령 주치의로 처음 발탁된 연세대 의대 출신 의사의 이름은?,1,False,허갑범,305,[“또 검찰 출신인가.” 박근혜 대통령이 지난 22일 안대희 전 대법관(사법시험 1...
298,황창규 KT 회장 내정자 첫 메시지...“인사청탁 하면 처벌”,기업&CEO,hankyung,“인사청탁하면 처벌하겠다. 방만한 경영을 끝내겠다.” 황창규 KT 회장 내정자(사진...,계열사를 53개로 늘린 사람의 이름은?,2,False,이석채,292,[롯데그룹에서 신격호 총괄회장(93)의 지위는 절대적이다. 롯데백화점 임원들에게 ‘...


In [46]:
# 네거티브 샘플 임의 제거를 위한 함수.
# 4개의 원소를 가진 파이썬 리스트가 들어오면 랜덤으로 1개~4개의 원소를 제거한다.
def remove_random_elements(input_list):
    # 원본 리스트를 변경하지 않기 위해 복사본 생성
    working_list = input_list.copy()

    # 제거할 원소의 개수를 랜덤으로 선택 (1에서 4 사이)
    num_to_remove = random.randint(1, 4)

    # 선택된 개수만큼 원소 제거
    for _ in range(num_to_remove):
        if working_list:  # 리스트가 비어있지 않은 경우에만 제거
            index_to_remove = random.randint(0, len(working_list) - 1)
            working_list.pop(index_to_remove)

    return working_list

In [47]:
# 4개의 원소를 가진 파이썬 리스트가 들어오면 랜덤으로 1개~4개의 원소를 제거한다.
original_list = [1, 2, 3, 4]
result = remove_random_elements(original_list)
print(f"원본 리스트: {original_list}")
print(f"결과 리스트: {result}")
print(f"제거된 원소 개수: {len(original_list) - len(result)}")

원본 리스트: [1, 2, 3, 4]
결과 리스트: [1, 2, 3]
제거된 원소 개수: 1


In [49]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  # 네거티브 샘플이 포함된 리스트. 즉, negative_samples 열에서 임의로 1개~4개의 원소 제거
  removed_negative_samples = remove_random_elements(list(negative_samples))

  # 정답 문서와 살아남은 원소들을 결합.
  documents = [context] + list(removed_negative_samples)

  # 문서의 순서를 섞는다.
  random.shuffle(documents)

  # 이제 samples에는 1~4개 사이의 원소를 갖고 있는 리스트들이 들어있다.
  # 이 1~4개 사이의 원소 중에서 포지티브 문서는 무조건 존재한다.
  samples.append(documents)

In [50]:
len(samples)

300

In [52]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:취득세 감면 종료에 여름 비수기까지 겹치면서 서울 재건축 단지들이 ‘거래 급감’과 ‘가격 하락’이라는 이중고를 겪는 가운데서도 잠실 주공5단지 등 일부 단지는 가격이 급등해 눈길을 끈다. 이들 단지의 가격 상승은 재건축 추진 본격화 등 ‘반짝 호재’여서 전체 재건축 시장에 미치는 파급효과가 그리 크지 않을 것이라는 게 부동산 업계의 분석이다.5일 부동산 업계에 따르면 서울 잠실동 ‘주공5단지’ 전용 77㎡는 이달 들어 10억4000만원에 시세가 형성됐다. 취득세 감면 종료 여파로 지난 6월 말 9억6000만원까지 떨어졌던 것과 비교하면 한 달 새 8000만원가량 오른 셈이다. 인근 잠실박사 공인 대표는 “취득세 감면 혜택이 끝났지만 지난달 7건의 거래가 이뤄졌다”며 “일부 집주인들이 급매물을 거둬들이고 있다”고 설명했다.잠실 주공5단지의 집값 오름세는 지난 3일 치러진 추진위원장 선거 효과 때문이다. 주공5단지는 올해 초 추진위원장이 법원에서 직무집행 정지 가처분 결정을 받으면서 재건축 사업이 난항을 겪어왔다. 하지만 이번에 새 추진위원장이 선출되면서 재건축에 탄력이 붙을 것이라는 전망이 가격을 끌어올리고 있다. 석봉수 주공5단지 추진위원은 “직무집행 정지됐던 위원장이 다시 선출됨에 따라 연말까지 조합설립 총회까지 마칠 수 있게 됐다”고 말했다.조합 해산을 두고 갈등을 겪었던 서울 길동 ‘신동아1·2차’도 최근 재건축 기대심리가 살아난 아파트다. 이 단지는 지난해 일부 주민이 “재건축 주민 분담금이 너무 높다”며 비상대책위원회를 만들고 조합해산에 나서면서 갈등이 커졌다. 하지만 조합이 최근 설계 변경을 통해 분담금을 낮추기로 비대위와 합의하면서 재건축 불확실성이 해소됐다. 길동 S공인 관계자는 “최근 전용 73㎡가 약 3억원으로 한 달 새 1000만~2000만원 정도 상승했다”고 말했다.
---


In [54]:
user_prompts = []
for sample in samples:
  user_prompt = '검색 결과:\n'
  for idx, s in enumerate(sample):
    user_prompt = user_prompt  + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [55]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(df['question'].to_list(), user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [56]:
print(user_prompt_for_answer[0])

질문:유아용 전동자동차와 장난감 레고 세트를 경품 추첨으로 주는 아파트는?
검색 결과:
문서1:17일 오전 서울 지하철 8호선 장지역 인근 ‘송파 파크하비오 푸르지오’ 모델하우스. 아파트와 오피스텔, 스트리트형 상가, 공연장 등이 들어서는 강남권 미니 신도시급 복합단지인 이곳을 찾은 방문객들이 100m가 넘는 긴 줄을 이뤘다. 시행사인 다함하비오는 주말 사흘 동안 방문객을 5만2000여명으로 집계했다. ‘위례2차 아이파크’ ‘래미안 강동 팰리스’ ‘마포한강2차 푸르지오’ 등 주말에 문을 연 서울과 수도권 주요 모델하우스 앞에도 30~40m의 줄이 늘어서는 등 방문객들로 북적였다. 생애최초 주택 구입자는 연말까지 아파트를 분양받으면 ‘취득세 면제’ ‘양도세 5년간 면제’ 등의 혜택을 볼 수 있어 수요자들이 모델하우스로 몰려들고 있다. 주말에 문을 연 전국의 10여개 모델하우스에 약 20만명이 찾았다고 건설사들은 밝혔다. 이번주(18~24일) 청약 물량은 1만5606가구에 이른다. 서울과 수도권 분양 아파트들은 대단지인 데다 입지여건이 좋아 수요자들의 관심이 높다. 19일에는 대우건설이 서울 문정지구에서 ‘송파 파크하비오 푸르지오’를 공급한다. 아파트(전용 84~151㎡) 999가구, 오피스텔(전용 23~74㎡) 2283실로 이뤄졌다. 3.3㎡당 아파트는 1690만원, 오피스텔은 1000만원 선으로 분양가가 주변 시세보다 싸게 책정됐다. 현대산업개발은 오는 20일 서울 송파구 위례신도시 C1-2블록에서 ‘위례2차 아이파크’(전용 90~115㎡) 청약을 받는다. 총 495가구에 평균 분양가는 3.3㎡당 1740만원대다. 21일에는 대우건설이 서울 합정동 ‘마포한강2차 푸르지오’(전용 83~110㎡)를 공급한다. 일반분양 물량은 총 198가구, 분양가는 3.3㎡당 1900만원대다. 이어 22일에는 삼성물산이 서울 천호동에서 ‘래미안 강동팰리스’(전용 59~155㎡) 청약에 나선다. 총 999가구로 평균 분양가는 3.3㎡당 1650만원대다. LH(한국토지주택공사)는 김

In [57]:
system_prompt = """당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번, 5번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]], [[ref5]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 답변은 최대한 풍부하게 작성해주시기 바랍니다.
10. 검색 결과는 모두 독립적인 문서들이며 반드시 연관되어져 있거나 순서대로 이어지는 문서들이 아닙니다.
"""

In [59]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:09<00:00,  1.83s/it]


In [60]:
result_lst

["유아용 전동자동차와 장난감 레고 세트를 경품으로 제공하는 아파트는 '김포한강신도시 반도유보라4차'입니다. 이 아파트는 모델하우스를 여는 반도건설이 인터넷 홈페이지를 통해 경품 추첨을 진행하며, 7일까지 퀴즈를 풀고 전화번호 등을 입력한 관심 고객 중 일부에게 유아용 전동자동차(1대)와 장난감 레고 세트(5개)를 제공합니다[[ref2]].",
 '2009년 이혼 소송 접수 건수는 4만7907건으로, 이는 글로벌 금융위기 직후의 정점에 해당합니다. 이후 이혼 소송 건수는 전체적으로 감소하는 추세를 보이고 있습니다[[ref2]].',
 "규제로 인해 중소 프랜차이즈 매장이 증가하고 있는 것으로 나타났습니다. 대기업 계열의 빵집 확장이 제한되면서, 중소 프랜차이즈들이 매장 확장에 속도를 내고 있습니다. 예를 들어, 영화배우 정준호가 주주인 '잇브레드'는 창업 1년여 만에 매장을 70여 곳으로 늘렸고, 저가 정책을 앞세운 '이지바이'도 매장을 크게 증가시켰습니다. 반면, 대기업 계열의 빵집들은 매장 수가 거의 변동이 없거나 미미한 증가에 그쳤습니다. 이러한 현상은 규제가 중소 프랜차이즈의 성장을 촉진하고 있다는 것을 보여줍니다[[ref3]]. \n\n또한, 일본의 100엔 숍 업체들도 소비세 인상 이후 저가 상품에 대한 수요 증가를 반영하여 점포를 크게 늘리고 있습니다. 일본의 4대 100엔숍 기업이 올해 450개가량의 신규 점포를 계획하고 있으며, 이는 지난해보다 증가한 수치입니다[[ref2]]. \n\n이와 같이 규제는 대기업의 확장을 제한하고, 중소 매장이나 저가 매장들이 증가하는 결과를 초래하고 있습니다.",
 '조텍컵에서 입상하면 상금은 페이팔(PayPal)을 통해 지급됩니다. 따라서 대회 신청 시 페이팔 계정을 필수로 기입해야 합니다[[ref2]].',
 '직무부트캠프를 제공하는 회사인 (주)코멘토의 대표는 이재성입니다. 이재성 대표는 직무부트캠프의 높은 만족도와 학생들에게 제공되는 기회에 대해 언급하며 프로그램의 중요성을 강조했습니다 [[ref1]].'

In [61]:
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 300/300 [10:31<00:00,  2.11s/it]


In [62]:
df['search_result'] = samples

In [63]:
df['answer'] = result_lst

In [64]:
extracted_ref_numbers = []
for sample in result_lst:
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [65]:
df['extracted_ref_numbers'] = extracted_ref_numbers

In [66]:
# 인용 실패한 데이터 제외
cited_df = df[~(df['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

In [67]:
print('답변이 존재하는 데이터:', len(cited_df))

답변이 존재하는 데이터: 288


In [68]:
cited_df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,search_result,answer,extracted_ref_numbers
0,어린이날 실수요자 끌기 … TV·車까지 경품 푸짐,부동산,hankyung,"‘자동차, 곡면 TV, 레고 세트….’ 가정의 달 5월을 맞아 아파트를 분양 중인 ...",유아용 전동자동차와 장난감 레고 세트를 경품 추첨으로 주는 아파트는?,2,False,‘김포한강신도시 반도유보라4차’,372,[유통업계가 황금 연휴(1~5일)로 시작하는 ‘가정의 달’ 5월을 맞아 대규모 세일...,[17일 오전 서울 지하철 8호선 장지역 인근 ‘송파 파크하비오 푸르지오’ 모델하우...,유아용 전동자동차와 장난감 레고 세트를 경품으로 제공하는 아파트는 '김포한강신도시 ...,[2]
1,이혼 건수 늘었는데...소송 줄어든 이유는,종합,hankyung,최근 들어 이혼 건수는 소폭 늘어나고 있지만 이혼 소송은 오히려 줄어들고 있는 것으...,2009년 이혼 소송 접수 건수는 몇 건인가?,1,False,4만7907건,318,[여성의 경제력과 사회적 지위가 높아지면서 이혼이나 사별(死別) 경험이 있는 여성의...,[여성의 경제력과 사회적 지위가 높아지면서 이혼이나 사별(死別) 경험이 있는 여성의...,"2009년 이혼 소송 접수 건수는 4만7907건으로, 이는 글로벌 금융위기 직후의 ...",[2]
2,대기업 규제해도 동네 빵집 안늘어,생활경제,hankyung,"대기업 계열 빵집 확장을 제한한 결과, 중소 프랜차이즈는 약진하는 반면 정작 ‘동네...",규제로 인해 어떤 종류의 매장이 증가했는가?,1,False,중소 프랜차이즈,22,[주요 유통 및 필수 소비재 업체들이 여기저기서 허리띠를 졸라매고 있다. 무분별하게...,[주요 유통 및 필수 소비재 업체들이 여기저기서 허리띠를 졸라매고 있다. 무분별하게...,규제로 인해 중소 프랜차이즈 매장이 증가하고 있는 것으로 나타났습니다. 대기업 계열...,"[2, 3]"
3,조텍컵 오버워치 아시아 커뮤니티 참가 모집,게임,acrofan,지포스 그래픽카드 전문 기업 조텍(ZOTAC Technology)이 남녀노소 누구나...,조텍컵에서 입상하면 어떤 플랫폼을 통해 상금을 받을 수 있는가?,1,False,페이팔,484,"[조텍코리아(ZOTAC Korea, 대표이사 김성표)가 엔비디아와 함께 차세대 그래...","[조텍코리아(ZOTAC Korea, 대표이사 김성표)가 엔비디아와 함께 차세대 그래...",조텍컵에서 입상하면 상금은 페이팔(PayPal)을 통해 지급됩니다. 따라서 대회에 ...,[2]
4,"(주)코멘토, 서울과기대에 직무부트캠프 제공",사회,acrofan,(주)코멘토가 유망 분야 직무에 대한 직무부트캠프를 서울과학기술대학(총장 이동훈)과...,직무부트캠프를 제공하는 회사를 운영하는 사람의 이름은?,2,False,이재성,595,[“구글이 꿈의 직장? 실력 없으면 못 버틴다”최근 구글 본사에서 일하는 한국인 개...,[(주)코멘토가 유망 분야 직무에 대한 직무부트캠프를 서울과학기술대학(총장 이동훈)...,직무부트캠프를 제공하는 회사인 (주)코멘토의 대표는 이재성입니다. 이재성 대표는 직...,[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,경단녀에게 이만한 직장 없죠”,종합,hankyung,“영어교육학과를 졸업했지만 기간제 교사로 일하는 것보다 하루 6시간 방과후학교 업무...,여선영씨와 매일 출퇴근을 함께 하는 사람은?,1,False,아들,232,[남덕우 전 국무총리 빈소가 차려진 서울 일원동 삼성서울병원 장례식장엔 19일 아침...,[“영어교육학과를 졸업했지만 기간제 교사로 일하는 것보다 하루 6시간 방과후학교 업...,여선영 씨와 매일 출퇴근을 함께 하는 사람은 정희숙 씨입니다. 정희숙 씨는 여선영 ...,[1]
284,침팬지도 나폴레옹·마키아벨리 못지않게 전략 짠다,책마을,hankyung,현대 사회에서 전략은 일상 용어다. 국가나 대기업이 사활을 걸 정도로 중요한 결정일...,전략의 역사'의 저자는 본래 어떤 학문을 연구하는 전문가인가?,1,False,전쟁연구학,223,[≪전쟁론≫ 이전에 전쟁 이론은 대부분 전략 및 전술 중심의 테마로 구성되었으나 ≪...,[≪전쟁론≫ 이전에 전쟁 이론은 대부분 전략 및 전술 중심의 테마로 구성되었으나 ≪...,"'전략의 역사'의 저자는 로렌스 프리드먼으로, 그는 런던 킹스칼리지 전쟁연구학부의 ...",[2]
285,대통령 주치의 이병석 교수,종합,hankyung,박근혜 대통령의 주치의로 이병석 연세대 의대 산부인과 교수(강남세브란스병원장·사진)...,대통령 주치의로 처음 발탁된 연세대 의대 출신 의사의 이름은?,1,False,허갑범,305,[“또 검찰 출신인가.” 박근혜 대통령이 지난 22일 안대희 전 대법관(사법시험 1...,[박근혜 대통령 당선인은 첫 장관들을 관료 교수 연구원 출신 전문가로 선택했다. 내...,대통령 주치의로 처음 발탁된 연세대 의대 출신 의사는 이병석 교수입니다. 그는 19...,[3]
286,황창규 KT 회장 내정자 첫 메시지...“인사청탁 하면 처벌”,기업&CEO,hankyung,“인사청탁하면 처벌하겠다. 방만한 경영을 끝내겠다.” 황창규 KT 회장 내정자(사진...,계열사를 53개로 늘린 사람의 이름은?,2,False,이석채,292,[롯데그룹에서 신격호 총괄회장(93)의 지위는 절대적이다. 롯데백화점 임원들에게 ‘...,[“인사청탁하면 처벌하겠다. 방만한 경영을 끝내겠다.” 황창규 KT 회장 내정자(사...,이름은 이석채 전 회장입니다. 그는 취임 전 30여 개였던 계열사를 53개로 늘린 ...,[1]


In [69]:
cited_df[['question', 'search_result', 'answer', 'extracted_ref_numbers']]

,question,search_result,answer,extracted_ref_numbers
0,유아용 전동자동차와 장난감 레고 세트를 경품 추첨으로 주는 아파트는?,[17일 오전 서울 지하철 8호선 장지역 인근 ‘송파 파크하비오 푸르지오’ 모델하우...,유아용 전동자동차와 장난감 레고 세트를 경품으로 제공하는 아파트는 '김포한강신도시 ...,[2]
1,2009년 이혼 소송 접수 건수는 몇 건인가?,[여성의 경제력과 사회적 지위가 높아지면서 이혼이나 사별(死別) 경험이 있는 여성의...,"2009년 이혼 소송 접수 건수는 4만7907건으로, 이는 글로벌 금융위기 직후의 ...",[2]
2,규제로 인해 어떤 종류의 매장이 증가했는가?,[주요 유통 및 필수 소비재 업체들이 여기저기서 허리띠를 졸라매고 있다. 무분별하게...,규제로 인해 중소 프랜차이즈 매장이 증가하고 있는 것으로 나타났습니다. 대기업 계열...,"[2, 3]"
3,조텍컵에서 입상하면 어떤 플랫폼을 통해 상금을 받을 수 있는가?,"[조텍코리아(ZOTAC Korea, 대표이사 김성표)가 엔비디아와 함께 차세대 그래...",조텍컵에서 입상하면 상금은 페이팔(PayPal)을 통해 지급됩니다. 따라서 대회에 ...,[2]
4,직무부트캠프를 제공하는 회사를 운영하는 사람의 이름은?,[(주)코멘토가 유망 분야 직무에 대한 직무부트캠프를 서울과학기술대학(총장 이동훈)...,직무부트캠프를 제공하는 회사인 (주)코멘토의 대표는 이재성입니다. 이재성 대표는 직...,[1]
...,...,...,...,...
283,여선영씨와 매일 출퇴근을 함께 하는 사람은?,[“영어교육학과를 졸업했지만 기간제 교사로 일하는 것보다 하루 6시간 방과후학교 업...,여선영 씨와 매일 출퇴근을 함께 하는 사람은 정희숙 씨입니다. 정희숙 씨는 여선영 ...,[1]
284,전략의 역사'의 저자는 본래 어떤 학문을 연구하는 전문가인가?,[≪전쟁론≫ 이전에 전쟁 이론은 대부분 전략 및 전술 중심의 테마로 구성되었으나 ≪...,"'전략의 역사'의 저자는 로렌스 프리드먼으로, 그는 런던 킹스칼리지 전쟁연구학부의 ...",[2]
285,대통령 주치의로 처음 발탁된 연세대 의대 출신 의사의 이름은?,[박근혜 대통령 당선인은 첫 장관들을 관료 교수 연구원 출신 전문가로 선택했다. 내...,대통령 주치의로 처음 발탁된 연세대 의대 출신 의사는 이병석 교수입니다. 그는 19...,[3]
286,계열사를 53개로 늘린 사람의 이름은?,[“인사청탁하면 처벌하겠다. 방만한 경영을 끝내겠다.” 황창규 KT 회장 내정자(사...,이름은 이석채 전 회장입니다. 그는 취임 전 30여 개였던 계열사를 53개로 늘린 ...,[1]


In [70]:
dataset = datasets.Dataset.from_pandas(cited_df)

In [71]:
dataset.push_to_hub("klue-mrc-original-question-with-1-to-4-negative-samples")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/longjae/klue-mrc-original-question-with-1-to-4-negative-samples/commit/7080ea49c070ba9c748903230f004ecb12f35600', commit_message='Upload dataset', commit_description='', oid='7080ea49c070ba9c748903230f004ecb12f35600', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/longjae/klue-mrc-original-question-with-1-to-4-negative-samples', endpoint='https://huggingface.co', repo_type='dataset', repo_id='longjae/klue-mrc-original-question-with-1-to-4-negative-samples'), pr_revision=None, pr_num=None)